In [1]:
%matplotlib inline

In [10]:
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
from matplotlib import rcParams

import numpy as np
import pandas as pd


import shapely.geometry as sgeom

import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

import dask.dataframe as dd
import dask.bag as db
import dask.diagnostics as dg

import geopandas as gpd


paper_path = Path('../../paper/figures/')

slide_path = Path('../../slides/figures/intro/')
proposal_path = Path('../../draft/figures/intro/')
#rcParams['font.family'] = 'Segoe Print'

In [61]:
YEAR = 2022
#url = f"https://noaa-ghcn-pds.s3.amazonaws.com/csv/by_year/{YEAR}.csv"
#url = f"https://noaa-ghcn-pds.s3.amazonaws.com/csv/by_year/{YEAR}.csv"
url = f'{YEAR}.csv'
names = ['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG', 'OBS-TIME']
ds = dd.read_csv(url, storage_options={'anon':True},  names=names, memory_map=False, 
                  dtype={'DATA_VALUE':'object'}, parse_dates=['DATE', 'OBS-TIME'])

C:\Users\story\miniconda3\envs\paper\Lib\site-packages\dask\dataframe\io\csv.py:594: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  head = reader(BytesIO(b_sample), nrows=sample_rows, **head_kwargs)


In [103]:
# {column name:extents of the fixed-width fields}
columns = {"ID": (0,11), "LATITUDE": (12, 20), "LONGITUDE": (21, 30), "ELEVATION": (31, 37),"STATE": (38, 40),
           "NAME": (41, 71), "GSN FLAG": (72, 75), "HCN/CRN FLAG": (76, 79),"WMO ID": (80, 85)}

In [104]:
df = pd.read_fwf("http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-stations.txt", 
                    colspecs=list(columns.values()), names=list(columns.keys())).dropna(subset=['STATE'])

In [105]:
df['STATE'].unique()

array(['AS', 'BH', 'BC', 'YT', 'NT', 'NU', 'AB', 'SK', 'MB', 'ON', 'QC',
       'NB', 'NS', 'PE', 'NL', 'MP', 'FM', 'GU', 'UM', 'PW', 'MH', 'PR',
       'SA', 'SD', 'CO', 'NE', 'AK', 'AL', 'AR', 'AZ', 'CA', 'TN', 'CT',
       'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY',
       'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND',
       'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
       'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY', 'VI'], dtype=object)

In [106]:
nydf = df[df['STATE'].str.match("NY")]

In [107]:
ds['ELEMENT'].unique().compute()

0     DATN
1     WSFG
2     DAPR
3     AWND
4     SX55
      ... 
6     RHMX
7     SX31
8     DASF
9     MDSF
10    WT07
Name: ELEMENT, Length: 74, dtype: string

In [68]:
nyds = ds[ds['ID'].isin(nydf['ID'].unique()) & ds['ELEMENT'].isin(['TAVG', 'PRCP' ])].compute()

In [69]:
ny = nyds.merge(nydf, on='ID')

In [70]:
ny.columns

Index(['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG',
       'OBS-TIME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'STATE', 'NAME',
       'GSN FLAG', 'HCN/CRN FLAG', 'WMO ID'],
      dtype='object')

In [71]:
ds['ELEMENT'].unique().compute()

0     DATN
1     WSFG
2     DAPR
3     AWND
4     SX55
      ... 
6     RHMX
7     SX31
8     DASF
9     MDSF
10    WT07
Name: ELEMENT, Length: 74, dtype: string

In [72]:
nyd = ny.pivot(index=['NAME','LATITUDE', 'LONGITUDE', 'DATE'], columns=['ELEMENT'], values=['DATA_VALUE']).reset_index()

In [73]:
nyd

NAME LATITUDE LONGITUDE  \
ELEMENT                                                      
0        (AVERILL PARK HIGH SCHOOL) AVE  42.6447  -73.5725   
1        (AVERILL PARK HIGH SCHOOL) AVE  42.6447  -73.5725   
2        (AVERILL PARK HIGH SCHOOL) AVE  42.6447  -73.5725   
3        (AVERILL PARK HIGH SCHOOL) AVE  42.6447  -73.5725   
4        (AVERILL PARK HIGH SCHOOL) AVE  42.6447  -73.5725   
...                                 ...      ...       ...   
63113                     YOUNGSTOWN 1S  43.2347  -79.0511   
63114                     YOUNGSTOWN 1S  43.2347  -79.0511   
63115                     YOUNGSTOWN 1S  43.2347  -79.0511   
63116                     YOUNGSTOWN 1S  43.2347  -79.0511   
63117                     YOUNGSTOWN 1S  43.2347  -79.0511   

                        DATE DATA_VALUE       
ELEMENT                            PRCP TAVG  
0        2022-04-07 00:00:00         18  NaN  
1        2022-04-08 00:00:00        411  NaN  
2        2022-04-12 00:00:00         46  NaN  
3        2022-04-13 00:00:00         13  NaN  
4        2022-04-14 00:00:00          0  NaN  
...                      ...        ...  ...  
63113    2022-05-13 00:00:00          0  NaN  
63114    2022-05-14 00:00:00          0  NaN  
63115    2022-05-15 00:00:00          0  NaN  
63116    2022-05-16 00:00:00          0  NaN  
63117    2022-05-17 00:00:00        102  NaN  

[63118 rows x 6 columns]

In [74]:
nyd.columns = [c[0] if i <4 else c[1] for i, c in enumerate(nyd.columns.to_flat_index())]

In [75]:
nyd.head()

,NAME,LATITUDE,LONGITUDE,DATE,PRCP,TAVG
0,(AVERILL PARK HIGH SCHOOL) AVE,42.6447,-73.5725,2022-04-07 00:00:00,18,NaN
1,(AVERILL PARK HIGH SCHOOL) AVE,42.6447,-73.5725,2022-04-08 00:00:00,411,NaN
2,(AVERILL PARK HIGH SCHOOL) AVE,42.6447,-73.5725,2022-04-12 00:00:00,46,NaN
3,(AVERILL PARK HIGH SCHOOL) AVE,42.6447,-73.5725,2022-04-13 00:00:00,13,NaN
4,(AVERILL PARK HIGH SCHOOL) AVE,42.6447,-73.5725,2022-04-14 00:00:00,0,NaN


In [76]:
nyd['TAVGF'] =  (nyd['TAVG'].astype(float)/10) *(9/5) + 32

In [77]:
nyd.head()

,NAME,LATITUDE,LONGITUDE,DATE,PRCP,TAVG,TAVGF
0,(AVERILL PARK HIGH SCHOOL) AVE,42.6447,-73.5725,2022-04-07 00:00:00,18,NaN,NaN
1,(AVERILL PARK HIGH SCHOOL) AVE,42.6447,-73.5725,2022-04-08 00:00:00,411,NaN,NaN
2,(AVERILL PARK HIGH SCHOOL) AVE,42.6447,-73.5725,2022-04-12 00:00:00,46,NaN,NaN
3,(AVERILL PARK HIGH SCHOOL) AVE,42.6447,-73.5725,2022-04-13 00:00:00,13,NaN,NaN
4,(AVERILL PARK HIGH SCHOOL) AVE,42.6447,-73.5725,2022-04-14 00:00:00,0,NaN,NaN


In [78]:
nyd['PRCP'].astype(float).describe()

count    62433.000000
mean        29.000080
std         68.666962
min          0.000000
25%          0.000000
50%          0.000000
75%         25.000000
max       1052.000000
Name: PRCP, dtype: float64

In [79]:
nyd['PRCPI'] = nyd['PRCP'].astype(float)/10 * 0.039370 

In [80]:
nyd['PRCPI'].describe()

count    62433.000000
mean         0.114173
std          0.270342
min          0.000000
25%          0.000000
50%          0.000000
75%          0.098425
max          4.141724
Name: PRCPI, dtype: float64

In [82]:
nyd.to_csv(f"nydata_{YEAR}.csv", index=False)

nyd

In [83]:
import homeomorphism_helpers as hh
import matplotlib.pyplot as plt

In [47]:
nygeo

,NAME,LATITUDE,LONGITUDE,DATE,PRCP,TAVG,TAVGF,PRCPI,geometry


In [85]:
mdf = pd.read_csv("nydata_2022.csv")
gdf = gpd.read_file('s_22mr22.zip')
    

In [86]:
nyshp = gdf[gdf['STATE'].str.match('NY')]
    nyg = mdf[(pd.to_datetime(mdf['DATE'], format='mixed').dt.strftime("%Y%m%d")== airports['map_date'])]
matching_dates = pd.to_datetime(mdf['DATE'], format='mixed').dt.strftime("%Y%m%d") == hh.airports['map_date']
nyg = mdf[( == hh.airports['map_date'])]
nygeo = gpd.GeoDataFrame(nyg, geometry=gpd.points_from_xy(nyg['LONGITUDE'], nyg['LATITUDE']), crs='EPSG:4269')

In [102]:
pd.to_datetime(mdf['DATE'], format='mixed').dt.strftime("%Y%m%d")

0        20220407
1        20220408
2        20220412
3        20220413
4        20220414
           ...   
63113    20220513
63114    20220514
63115    20220515
63116    20220516
63117    20220517
Name: DATE, Length: 63118, dtype: object

In [98]:
hh.airports['map_date']

'20220609'